In [ ]:
# # Required installations (run once)
# !brew install wget  # Added by Miles
# !pip install --upgrade --ignore-installed wrapt  # Added by Miles
# !pip install tensorflow==2.0.0-beta0  # Edited by Miles (switch to CPU version)
# !pip install tensorflow_datasets  # Added by Miles

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1
%aimport helper

## Import Libraries

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
keras = tf.keras

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import pymongo        # MongoDB

from PIL import Image

import helper

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from io import BytesIO
import boto3

from image_feature_extractor import ImageFeatureExtractor


### Mongo Keys

In [3]:
# Define path to secret
#secret_path = os.path.join(os.environ['HOME'], '.secret', 'mongo.json')

In [4]:
# keys = helper.get_keys(secret_path)
# mongo_user = keys['user_id']
# mongo_pw = keys['password']

## Data download

In [5]:
# from os import walk

image_paths = []
target_labels = []
for dirpath, dirnames, filenames in os.walk('downloads/all_photos'):
#for dirpath, dirnames, filenames in os.walk('test_data'):
    for ff in filenames:
        if ff[:1] != '.':
            curr_path = os.path.join('.',dirpath, ff)
            temp_name = dirpath[dirpath.rfind('/') + 1:]
#             print(curr_path)  
            target_labels.append(temp_name)
            image_paths.append(curr_path)
        
target_labels[:20]

['vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince',
 'vince']

In [6]:
image_paths[:20]

['./downloads/all_photos/vince/110.matt-schulzeactorwestwood-los-angeles-usa05082002lab7078-DKX8MR.jpg',
 './downloads/all_photos/vince/84.Matt+Schulze+outside+ArcLight+Theatre+Lp9hn7BIIGqs.jpg',
 './downloads/all_photos/vince/67.the-transporter-film-premiere-los-angeles-america-shutterstock-editorial-391154x.jpg',
 './downloads/all_photos/vince/28.mattschulze-torque-5.jpg',
 './downloads/all_photos/vince/179.Matt-in-Blade-II-matt-schulze-26020173-853-480.jpg',
 './downloads/all_photos/vince/140.matt-schulze-actor-scandal.png',
 './downloads/all_photos/vince/71.Matt-in-Torque-matt-schulze-23059204-700-525.jpg',
 './downloads/all_photos/vince/5.7783020.jpg',
 './downloads/all_photos/vince/51.matt-schulze-kiss-of-the-dragon-film-prem-century-city-los-angeles-H82NWB.jpg',
 './downloads/all_photos/vince/17.Matt-in-Torque-matt-schulze-23058761-700-525.jpg',
 './downloads/all_photos/vince/21.depositphotos_17572761-stock-photo-matt-schulze.jpg',
 './downloads/all_photos/vince/63.Matt-in-The-T

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
])

def extract_features(image):
    """Return a vector of 1280 deep features for image."""
    image_resized = prepare_image(image)
    image_np = image_resized.numpy()
    images_np = np.expand_dims(image_np, axis=0)
    image_np.shape, images_np.shape
    deep_features = model.predict(images_np)
    return deep_features[0]

extract_features(image)

### Get Features

In [7]:
#%pdb

In [8]:
forest = RandomForestClassifier(n_estimators=10, n_jobs=-1)
extractor = ImageFeatureExtractor()

In [9]:
pipe = Pipeline([
    ('extract_deep_features', extractor),
    ('classify', forest)
])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, target_labels)

In [11]:
X_train[0]

'./downloads/all_photos/megan/36.large533cb2969923d@2x.jpg'

In [12]:
pipe.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +
/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9 bytes but only got 8. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 6. Skipping tag 18
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Pipeline(memory=None,
     steps=[('extract_deep_features', ImageFeatureExtractor(height=160,
           model=<tensorflow.python.keras.engine.sequential.Sequential object at 0x1a308656a0>,
           width=160)), ('classify', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=No..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [13]:
png = ['./downloads/all_photos/dominic/118.fate-of-the-furious-banner-6.png']

In [14]:
pipe.predict(X_test)

/anaconda3/lib/python3.7/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


array(['brixton', 'brian', 'brian', ..., 'deckard', 'gisele', 'brian'],
      dtype='<U7')

In [15]:
pipe.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


0.32556270096463025

In [27]:
#import webbrowser

# !cd downloads/public_photos
# !wget http://www.sosia.biz/files/immagini/1289210942-DSCF0851.JPG
# !ls

vinny = ['http://www.sosia.biz/files/immagini/1289210942-DSCF0851.JPG']
laura = ['https://cdn-images-1.medium.com/max/1200/1*jM7PrjvG20306cXwjgN6hA@2x.jpeg']
mia = ['https://media.licdn.com/dms/image/C4E03AQEUu7pgy0zqrw/profile-displayphoto-shrink_200_200/0?e=1563408000&v=beta&t=IQESr0Ho16othgeTRgp0nrGlXRkv6c-WiSHf_nCzRlk']
werlindo = ['https://cdn-images-1.medium.com/max/1200/2*T33SKqm3ldv2QkTE3QQ0Dw.jpeg']

In [28]:
vinny

['http://www.sosia.biz/files/immagini/1289210942-DSCF0851.JPG']

In [29]:
pipe.predict(werlindo)

array(['roman'], dtype='<U7')

# DELETE EVERYTHING BELOW 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
import boto3
from PIL import Image
import numpy as np
from io import BytesIO
import requests


In [ ]:
    
    def __init__(self, model="MobileNetV2", height=160, width=160):
        """Creates an ImageFeatureExtractor using the specified model."""
        self.height, self.width = height, width
        if model == "MobileNetV2":
            base_model = tf.keras.applications.MobileNetV2(
                input_shape=(height, width, 3),
               include_top=False,
               weights='imagenet'
            )
        else:
            raise Exception("Model unknown")
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        self.model = tf.keras.Sequential([base_model, global_average_layer])

In [ ]:
    def fit(self, X, y):
        """We're using a pre-trained model, so there's nothing to fit."""
        return self

In [ ]:
    def transform(self, X):
        """Transforms image file paths into Numpy arrays of deep features."""
        result = []
        for image_pathname in X:
            result.append(self._transform_one(image_pathname))
        return result

In [ ]:
def transform_one(image_pathname):
    """Transforms a single image pathname into deep features."""
    try:
        img = Image.open(image_pathname)
    except:
        response = requests.get(image_pathname)
        img = Image.open(BytesIO(response.content))
    img.load()
    img = img.convert("RGB")
    image = np.asarray(img)
    image = image[:,:,:3]
    return image.shape

In [ ]:
transform_one(png[0])

In [ ]:
test6 = np.asarray(png)
test6

In [ ]:
    def _prepare_image(self, image):
        """Converts an image to the expected format for prediction."""
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        image = tf.image.resize(image, (self.height, self.width))
        return image

In [ ]:
    def _extract_features(self, image):
        """Return a vector of 1280 deep features for image."""
        image_resized = self._prepare_image(image)
        image_np = image_resized.numpy()
        images_np = np.expand_dims(image_np, axis=0)
        image_np.shape, images_np.shape
        deep_features = self.model.predict(images_np)
        return deep_features[0]

In [ ]:
    def fetch_image_from_s3(self, bucket, key):            
        """Fetches an image from S3 and returns a numpy array."""
        s3 = boto3.client('s3')
        response = s3.get_object(Bucket=bucket, Key=key)
        body = response['Body']
        data = body.read()    
        f = BytesIO(data)
        image = Image.open(f)   
        image_data = np.asarray(image)
        return image_data

In [ ]:
x = np.zeros( (106, 106, 3) )
result = x[:, :, :2]
print(result.shape)

# STOP DELETING

In [ ]:
# Instantiate Class
extractor = ImageFeatureExtractor()

In [ ]:
# Store Features - list of arrays
features = extractor.transform(image_paths)

In [ ]:
# Turn into list of lists because easier with MongoDB
features_list = [feature.tolist() for feature in features]

In [ ]:
# Zip them so can iterate through them
zipped_imgs = list(zip(image_paths,features_list))

In [ ]:
# Create list of dictionaries; so can be ingested by MongoDB
list_of_dicts = [{'url': img[0], 'features':img[1]} for img in zipped_imgs]

### Upload results to MongoDB

In [ ]:
# Instantiate client
client = pymongo.MongoClient("mongodb+srv://" + mongo_user + ":" 
                         + mongo_pw 
                         + "@dsaf-oy1s0.mongodb.net/test?retryWrites=true")


In [ ]:
# Get DB, Collection
db = client['furious']
coll = db['images']

In [ ]:
# Wipe collection to start fresh
coll.delete_many({})

In [ ]:
# Insert Results
coll.insert_many(list_of_dicts)

In [ ]:
features_returned = [np.array(x['features']) for x in coll.find()]

In [ ]:
features_returned

# LSM Random Forests

In [ ]:
target = target_labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    target, 
                                                    test_size = 0.25,
                                                    random_state=123)

### Regular Tree

In [ ]:
tree_clf = DecisionTreeClassifier(criterion = "gini", max_depth = 5) 
tree_clf.fit(X_train, y_train)

In [ ]:
tree_clf.feature_importances_

In [ ]:
pred = tree_clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

### Random Forest

In [ ]:
forest = RandomForestClassifier(n_estimators=20)
forest.fit(X_train, y_train)

In [ ]:
forest.score(X_train, y_train)

In [ ]:
forest.score(X_test, y_test)

In [ ]:
forest.predict(X_train)

#### This is likely not right.... 

In [ ]:
len(features_returned)

![](https://images.unsplash.com/photo-1518137569197-67c41a95d8de?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2560&q=80)

# Graveyard

----
### _We can do train / test split before we push through classification models?_

In [ ]:
train_images = [(img.numpy(), label.numpy())
                for (img, label) in raw_train.take(10000)]

In [ ]:
test_images = [(img.numpy(), label.numpy())
               for (img, label) in raw_test.take(1000)]

In [ ]:
plt.imshow(train_images[0][0])
plt.title(get_label_name(train_images[0][1]))

In [ ]:
train_X_img, train_y = zip(*train_images)
test_X_img, test_y = zip(*test_images)

# Extract features from some images

In [ ]:
TRAIN_LIMIT = 1000
TEST_LIMIT = 200

In [ ]:
train_X_small = [extract_features(img)
                 for img in train_X_img[:TRAIN_LIMIT]]
test_X_small = [extract_features(img)
                for img in test_X_img[:TEST_LIMIT]]

In [ ]:
train_y_small = train_y[:TRAIN_LIMIT]
test_y_small = test_y[:TEST_LIMIT]

# Logistic Regression on Deep Features

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
lr.fit(train_X_small, train_y_small)

In [ ]:
train_preds = lr.predict(train_X_small)
(sum(train_y_small) / len(train_y_small),
 sum(train_preds == train_y_small) / len(train_y_small))

In [ ]:
test_preds = lr.predict(test_X_small)
(sum(test_y_small) / len(test_y_small),
 sum(test_preds == test_y_small) / len(test_y_small))

In [ ]:
idxs=np.argsort(np.abs(lr.coef_[0]))[::-1]

In [ ]:
plt.hist(lr.coef_[0])

In [ ]:
import pandas as pd
pd.DataFrame({'values': lr.coef_[0][idxs], 'idx': idxs})

# Random Forest on Deep Features

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
rfc.fit(train_X_small, train_y_small)

In [ ]:
train_preds = rfc.predict(train_X_small)
(sum(train_y_small) / len(train_y_small),
 sum(train_preds == train_y_small) / len(train_y_small))

In [ ]:
test_df = pd.DataFrame(test_X_small)

In [ ]:
test_df.head()

In [ ]:
test_preds = rfc.predict(test_X_small)
(sum(test_y_small) / len(test_y_small),
 sum(test_preds == test_y_small) / len(test_y_small))

In [ ]:
rf_idxs = np.argsort(rfc.feature_importances_)[::-1]

In [ ]:
rf_idxs

In [ ]:
rfc.feature_importances_[rf_idxs]

In [ ]:
permuted_test_df = test_df.copy()
idx_list = rf_idxs[:10]
for idx in idx_list:
    important_feature = test_df.loc[:, idx].copy().values
    np.random.shuffle(important_feature)
    permuted_test_df.loc[:, idx] = important_feature

In [ ]:
test_preds = rfc.predict(permuted_test_df)
(sum(test_y_small) / len(test_y_small),
 sum(test_preds == test_y_small) / len(test_y_small))

##### Look at DB names
cur = client.list_databases()

for item in cur:
    print(item)

###### Look at everything in our collection!
cur = coll.find({})

for item in cur:
    print(item)